In [4]:
! pip install z3-solver

In [1]:
from z3 import *
import json


In [9]:
def classify_mus(mus_literals, cell_condition_constraints, rule_constraints):
    mus_cell_condition_literals = [lit for lit in mus_literals if lit in map(str, cell_condition_constraints)]
    if mus_cell_condition_literals:
        return "Cell Condition"
    else:
        mus_rule_literals = [lit for lit in mus_literals if lit in map(str, rule_constraints)]
        if mus_rule_literals:
            return "Sudoku Rules"
        else:
            return "Unknown"

In [18]:
def main():
    X = [[Int("x_%s_%s" % (i, j)) for i in range(9)] for j in range(9)]
    instance = [
        [0,0,0,0,9,4,6,3,0],
        [0,0,0,5,1,0,0,0,7],
        [0,8,9,0,0,0,0,4,0],
        [0,0,0,0,0,0,2,0,8],
        [0,6,0,2,0,1,0,5,0],
        [1,0,2,0,0,0,0,0,0],
        [0,7,0,0,0,0,5,2,0],
        [9,0,0,0,6,5,0,0,0],
        [0,4,0,9,7,0,0,0,0]
    ]
    cells_c = [And(1 <= X[i][j], X[i][j] <= 9) for i in range(9) for j in range(9)]
    rows_c = [Distinct(X[i]) for i in range(9)]
    cols_c = [Distinct([X[i][j] for i in range(9)]) for j in range(9)]
    sq_c = [Distinct([X[3 * i0 + i][3 * j0 + j] for j in range(3) for i in range(3)])
            for j0 in range(3) for i0 in range(3)]
    instance_c = [If(instance[i][j] == 0, True, X[i][j] == instance[i][j]) for i in range(9) for j in range(9)]
    flip = [Not(X[0][2] == 5)]

    cell_condition_constraints = []
    rule_constraints = []


    # Separate cell condition and rule constraints
    for i in range(9):
        for j in range(9):
            if instance[i][j] != 0:
                cell_condition_constraints.append((If(False, True, X[i][j] == instance[i][j]), True)) # True flag for cell condition constraints

    for constraint in [cells_c, rows_c, cols_c, sq_c]:
        for c in constraint:
            rule_constraints.append((c, False)) # False flag for rule constraints
    

    constraints = cells_c + rows_c + cols_c + sq_c + instance_c + flip

    

    csolver = SubsetSolver(constraints)
    msolver = MapSolver(n=csolver.n)
    mus_cell_condition = []
    mus_rule = []
    for lits in enumerate_sets(csolver, msolver):
        mus_literals = lits[1]
        # Identify which constraints are cell conditions or rule constraints
        mus_cell_condition_literals = [lit for lit in mus_literals if lit in map(str, [c[0] for c in cell_condition_constraints])]
        mus_rule_literals = [lit for lit in mus_literals if lit in map(str, [c[0] for c in rule_constraints])]
        mus_cell_condition.append((mus_cell_condition_literals, True))  # True flag for cell condition MUSs
        mus_rule.append((mus_rule_literals, False))  # False flag for rule MUSs

    # Output MUSs to a JSON file
    mus_data = {
        "cell_condition_MUS": mus_cell_condition,
        "rule_MUS": mus_rule
    }
    with open("unsat_cores.json", "w") as f:
        json.dump(mus_data, f, indent=4)





def get_id(x):
    return Z3_get_ast_id(x.ctx.ref(),x.as_ast())


def MkOr(clause):
    if clause == []:
       return False
    else:
       return Or(clause)

class SubsetSolver:
   constraints = []
   n = 0
   s = Solver()
   varcache = {}
   idcache = {}

   def __init__(self, constraints):
       self.constraints = constraints
       self.n = len(constraints)
       for i in range(self.n):
           self.s.add(Implies(self.c_var(i), constraints[i]))

   def c_var(self, i):
       if i not in self.varcache:
          v = Bool(str(self.constraints[abs(i)]))
          self.idcache[get_id(v)] = abs(i)
          if i >= 0:
             self.varcache[i] = v
          else:
             self.varcache[i] = Not(v)
       return self.varcache[i]

   def check_subset(self, seed):
       assumptions = self.to_c_lits(seed)
       return (self.s.check(assumptions) == sat)
        
   def to_c_lits(self, seed):
       return [self.c_var(i) for i in seed]

   def complement(self, aset):
       return set(range(self.n)).difference(aset)

   def seed_from_core(self):
       core = self.s.unsat_core()
       return [self.idcache[get_id(x)] for x in core]

   def shrink(self, seed):       
       current = set(seed)
       for i in seed:
          if i not in current:
             continue
          current.remove(i)
          if not self.check_subset(current):
             current = set(self.seed_from_core())
          else:
             current.add(i)
       return current

   def grow(self, seed):
       current = seed
       for i in self.complement(current):
           current.append(i)
           if not self.check_subset(current):
              current.pop()
       return current



class MapSolver:
   def __init__(self, n):
       """Initialization.
             Args:
            n: The number of constraints to map.
       """
       self.solver = Solver()
       self.n = n
       self.all_n = set(range(n))  # used in complement fairly frequently

   def next_seed(self):
       """Get the seed from the current model, if there is one.
	
            Returns:
            A seed as an array of 0-based constraint indexes.
       """
       if self.solver.check() == unsat:
            return None
       seed = self.all_n.copy()  # default to all True for "high bias"
       model = self.solver.model()
       for x in model:
           if is_false(model[x]):
              seed.remove(int(x.name()))
       return list(seed)

   def complement(self, aset):
       """Return the complement of a given set w.r.t. the set of mapped constraints."""
       return self.all_n.difference(aset)

   def block_down(self, frompoint):
       """Block down from a given set."""
       comp = self.complement(frompoint)
       self.solver.add( MkOr( [Bool(str(i)) for i in comp] ) )

   def block_up(self, frompoint):
       """Block up from a given set."""
       self.solver.add( MkOr( [Not(Bool(str(i))) for i in frompoint] ) )
    


def enumerate_sets(csolver, map, max_iterations=10):
    """Basic MUS/MCS enumeration, as a simple example."""
    iteration = 0
    while iteration < max_iterations:
        seed = map.next_seed()
        if seed is None:
            return
        if csolver.check_subset(seed):
            MSS = csolver.grow(seed)
            yield ("MSS", [str(c) for c in csolver.to_c_lits(MSS)])  # Convert BoolRef objects to strings
            map.block_down(MSS)
        else:
            MUS = csolver.shrink(seed)
            yield ("MUS", [str(c) for c in csolver.to_c_lits(MUS)])  # Convert BoolRef objects to strings
            map.block_up(MUS)
        iteration += 1



main()


In [ ]:
# pankaj
# cells_c + rows_c + cols_c + sq_c + instance_c + flip
s = Solver()
sudoku_rules = cell_s + rows_s + cols_s + sq_s
constraints = []

for i in sudoku_rules:
    s.assert_and_track(i, "sudoku_rules_" + str(i))

for i in instance_c: # here instance_c is the complete solved sudoku (cell_condition_constraints)
    s.assert_and_track(i, "Value_cons_" + str(i))

res = s.check()
if res == z3.sat:
    print("Golmal hai")
    exit()
elif res == z3.unsat:
    print(s.unsat_core())